# Project: Decoding Molecules From Fingerprints.
## Group Members:
### Qi Chen, e-mail: gusqichr@student.gu.se
### Nils Dunlop, e-mail: gusdunlni@student.gu.se
### Francisco Alejandro Erazo Piza, e-mail: guserafr@student.gu.se
***

In [83]:
import os
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
from sklearn.manifold import MDS
import numpy as np
import matplotlib.pyplot as plt
import warnings

In [84]:
# Set user directory and project root
USER_DIR = os.path.expanduser('~')
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Define the input directories for both sets of data
INPUT_DIR_1 = os.path.join(PROJECT_ROOT, 'data/bit_flipping_nn')
INPUT_DIR_2 = os.path.join(PROJECT_ROOT, 'data/bit_flipping_nn_review_2')

In [87]:
def load_molecule_dataframes(input_dir, flip_type="random_flips"):
    molecules = {
        'parecoxib': [],
        'celecoxib': [],
        'cimicoxib': [],
        'deracoxib': [],
        'anitrazafen': []
    }
    
    if flip_type == "single_flip":
        # For single-flipped files
        for molecule in molecules.keys():
            filename = f'{molecule.lower()}_flipped_df.parquet'
            filepath = os.path.join(input_dir, filename)
            
            try:
                df = pd.read_parquet(filepath)
                molecules[molecule].append({
                    'n_flips': 1,  # These are single-flipped molecules
                    'df': df
                })
                print(f"Loaded {molecule} with 1 flip: {df.shape}")
            except Exception as e:
                print(f"Error loading {filename}: {e}")
    else:
        # For multi-flipped files
        flip_counts = [2, 4, 8, 128, 1024]
        for molecule in molecules.keys():
            for n_flips in flip_counts:
                filename = f'{molecule}_df_{n_flips}_{flip_type}.parquet'
                filepath = os.path.join(input_dir, filename)
                
                try:
                    df = pd.read_parquet(filepath)
                    molecules[molecule].append({
                        'n_flips': n_flips,
                        'df': df
                    })
                    print(f"Loaded {molecule} with {n_flips} flips: {df.shape}")
                except Exception as e:
                    print(f"Error loading {filename}: {e}")
    
    return molecules

In [88]:
# Load both types of data
print("Loading single-flipped data...")
single_flipped = load_molecule_dataframes(INPUT_DIR_1, "single_flip")

print("\nLoading multi-flipped data...")
multi_flipped = load_molecule_dataframes(INPUT_DIR_2, "random_flips")

# Combine the datasets
combined_molecules = {}
for molecule in single_flipped:
    combined_molecules[molecule] = single_flipped[molecule] + multi_flipped[molecule]

Loading single-flipped data...
Loaded parecoxib with 1 flip: (2048, 9)
Loaded celecoxib with 1 flip: (2048, 9)
Loaded cimicoxib with 1 flip: (2048, 9)
Loaded deracoxib with 1 flip: (2048, 9)
Loaded anitrazafen with 1 flip: (2048, 9)

Loading multi-flipped data...
Loaded parecoxib with 2 flips: (1024, 9)
Loaded parecoxib with 4 flips: (512, 9)
Loaded parecoxib with 8 flips: (256, 9)
Loaded parecoxib with 128 flips: (16, 9)
Loaded parecoxib with 1024 flips: (2, 9)
Loaded celecoxib with 2 flips: (1024, 9)
Loaded celecoxib with 4 flips: (512, 9)
Loaded celecoxib with 8 flips: (256, 9)
Loaded celecoxib with 128 flips: (16, 9)
Loaded celecoxib with 1024 flips: (2, 9)
Loaded cimicoxib with 2 flips: (1024, 9)
Loaded cimicoxib with 4 flips: (512, 9)
Loaded cimicoxib with 8 flips: (256, 9)
Loaded cimicoxib with 128 flips: (16, 9)
Loaded cimicoxib with 1024 flips: (2, 9)
Loaded deracoxib with 2 flips: (1024, 9)
Loaded deracoxib with 4 flips: (512, 9)
Loaded deracoxib with 8 flips: (256, 9)
Loaded

In [95]:
def analyze_unique_smiles(molecules_dict):
    """
    Analyzes unique SMILES strings for each molecule and flip count.
    Returns a dictionary with statistics about unique SMILES per molecule/flip combination.
    """
    results = {}
    
    for molecule_name, molecule_data in molecules_dict.items():
        results[molecule_name] = {}
        
        for data in molecule_data:
            n_flips = data['n_flips']
            df = data['df']
            
            # Get unique SMILES and their counts
            unique_smiles = df['Generated_SMILES'].value_counts()
            
            results[molecule_name][n_flips] = {
                'total_predictions': len(df),
                'unique_smiles_count': len(unique_smiles),
                'most_common_smiles': unique_smiles.head(5).to_dict(),
                'unique_ratio': len(unique_smiles) / len(df) * 100
            }
    
    return results

def display_analysis(analysis_results):
    print("\n=== SMILES Analysis ===")
    
    for molecule, flip_data in analysis_results.items():
        print(f"\nMolecule: {molecule}")
        
        # Sort by number of flips to display in order
        for n_flips, stats in sorted(flip_data.items()):
            print(f"\n  Flips: {n_flips}")
            print(f"  Total predictions: {stats['total_predictions']}")
            print(f"  Unique SMILES: {stats['unique_smiles_count']}")
            print(f"  Unique ratio: {stats['unique_ratio']:.2f}%")
            print("  Top 5 most common SMILES:")
            for smiles, count in stats['most_common_smiles'].items():
                print(f"    Count: {count} - {smiles[:50]}...")

In [96]:
analysis_results = analyze_unique_smiles(combined_molecules)
display_analysis(analysis_results)


=== SMILES Analysis ===

Molecule: parecoxib

  Flips: 1
  Total predictions: 2048
  Unique SMILES: 8
  Unique ratio: 0.39%
  Top 5 most common SMILES:
    Count: 2035 - CCC(=O)NS(=O)(=O)c1ccc(-c2c(-c3ccccc3)noc2C)cc1...
    Count: 6 - CCC(=O)NS(=O)(=O)c1ccc(-c2noc(C)c2-c2ccccc2)cc1...
    Count: 2 - Cc1onc(-c2ccccc2)c1-c1ccc(S(=O)(=O)NC(=O)CC(=O)NS(...
    Count: 1 - CCC(=O)NS(=O)(=O)c1ccc(-c2c(-c3noc(C)c3-c3ccccc3)n...
    Count: 1 - CCC(=O)NS(=O)(=O)c1ccc(-c2noc(C)c2-c2c(-c3ccccc3)n...

  Flips: 2
  Total predictions: 1024
  Unique SMILES: 8
  Unique ratio: 0.78%
  Top 5 most common SMILES:
    Count: 1011 - CCC(=O)NS(=O)(=O)c1ccc(-c2c(-c3ccccc3)noc2C)cc1...
    Count: 6 - CCC(=O)NS(=O)(=O)c1ccc(-c2noc(C)c2-c2ccccc2)cc1...
    Count: 2 - Cc1onc(-c2ccccc2)c1-c1ccc(S(=O)(=O)NC(=O)CC(=O)NS(...
    Count: 1 - CCC(=O)NS(=O)(=O)c1ccc(S(=O)(=O)NC(=O)c2c(-c3ccccc...
    Count: 1 - CCC(=O)NS(=O)(=O)c1ccc(-c2noc(C)c2-c2c(-c3ccccc3)n...

  Flips: 4
  Total predictions: 512
  Unique SMILES: 11

### Analyze MolWeight and Atom Count when bits are flipped
---

In [97]:
def calculate_molecular_properties(molecules_dict):
    """
    Adds molecular weight and atom count columns to each dataframe in the molecules dictionary
    """
    for molecule_name, molecule_data in molecules_dict.items():
        print(f"\nProcessing {molecule_name}:")
        
        for data in sorted(molecule_data, key=lambda x: x['n_flips']):  # Sort by number of flips
            df = data['df']
            n_flips = data['n_flips']
            
            # Initialize new columns
            df['mol_weight'] = None
            df['atoms'] = None

            # Calculate properties for each SMILES string
            valid_count = 0
            for idx, row in df.iterrows():
                try:
                    if row['Generated_SMILES'] == 'Invalid SMILES string':
                        continue
                        
                    mol = Chem.MolFromSmiles(row['Generated_SMILES'])
                    if mol is not None:
                        df.at[idx, 'mol_weight'] = Descriptors.ExactMolWt(mol)
                        df.at[idx, 'atoms'] = mol.GetNumAtoms()
                        valid_count += 1
                    else:
                        print(f"Warning: Could not parse SMILES for {molecule_name} (flips: {n_flips}) at index {idx}")
                except Exception as e:
                    print(f"Error processing {molecule_name} (flips: {n_flips}) at index {idx}: {e}")
            
            # Print summary statistics
            print(f"\n  {n_flips} flips:")
            print(f"  Total molecules: {len(df)}")
            print(f"  Valid molecules: {valid_count}")
            print(f"  Average molecular weight: {df['mol_weight'].mean():.2f}")
            print(f"  Average atom count: {df['atoms'].mean():.2f}")
    
    return molecules_dict

In [98]:
combined_molecules = calculate_molecular_properties(combined_molecules)


Processing parecoxib:

  1 flips:
  Total molecules: 2048
  Valid molecules: 2048
  Average molecular weight: 370.65
  Average atom count: 26.04

  2 flips:
  Total molecules: 1024
  Valid molecules: 1024
  Average molecular weight: 371.21
  Average atom count: 26.08

  4 flips:
  Total molecules: 512
  Valid molecules: 512
  Average molecular weight: 374.59
  Average atom count: 26.32

  8 flips:
  Total molecules: 256
  Valid molecules: 255
  Average molecular weight: 521.76
  Average atom count: 35.60

  128 flips:
  Total molecules: 16
  Valid molecules: 12
  Average molecular weight: 722.63
  Average atom count: 49.75

  1024 flips:
  Total molecules: 2
  Valid molecules: 0
  Average molecular weight: nan
  Average atom count: nan

Processing celecoxib:

  1 flips:
  Total molecules: 2048
  Valid molecules: 2048
  Average molecular weight: 381.11
  Average atom count: 26.00

  2 flips:
  Total molecules: 1024
  Valid molecules: 1024
  Average molecular weight: 381.14
  Average at

### Plot results
---

In [102]:
def plot_property_changes(molecules_data, flip_counts, suffix="", title_suffix=""):
    """
    Create a plot showing how molecular properties change with specific flip counts.
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    colors = ['b', 'g', 'r', 'c', 'm']
    
    for molecule_name in molecules_data:
        weights = []
        atoms = []
        valid_flips = []
        
        for n_flips in flip_counts:
            data = next((d for d in molecules_data[molecule_name] 
                        if d['n_flips'] == n_flips), None)
            if data:
                df = data['df']
                weights.append(df['mol_weight'].mean())
                atoms.append(df['atoms'].mean())
                valid_flips.append(n_flips)
        
        if weights:  # Only plot if we have data
            color = colors.pop(0)
            ax1.plot(valid_flips, weights, '-o', color=color, 
                    label=f"{molecule_name} (original: {weights[0]:.1f})")
            ax2.plot(valid_flips, atoms, '-o', color=color, 
                    label=f"{molecule_name} (original: {int(atoms[0])})")

    for ax, title, ylabel in zip(
        [ax1, ax2],
        [f'Change in Molecular Weight vs Number of Flips{title_suffix}',
         f'Change in Atom Count vs Number of Flips{title_suffix}'],
        ['Average Molecular Weight', 'Average Atom Count']
    ):
        ax.set_xlabel('Number of Flips')
        ax.set_ylabel(ylabel)
        ax.set_title(title)
        ax.grid(True, linestyle='--', alpha=0.7)
        ax.legend()
        ax.set_xticks(flip_counts)
        ax.set_xticklabels(flip_counts)

    plt.tight_layout()
    # Save to the bit_flipping_nn folder
    save_path = os.path.join(INPUT_DIR_1, f'property_changes{suffix}.png')
    plt.savefig(save_path)
    plt.close()

In [101]:
# Generate zoomed version (1,2,4,8)
plot_property_changes(
    combined_molecules, 
    [1, 2, 4, 8], 
    suffix="_zoomed",
    title_suffix=" (Zoomed)"
)

# Generate full version (1,2,4,8,128)
plot_property_changes(
    combined_molecules, 
    [1, 2, 4, 8, 128], 
    suffix="_full",
    title_suffix=" (Full Range)"
)

### Analyze unique weights and atoms
---

In [104]:
# For each molecule in combined molecules
for molecule_name in combined_molecules:
    print(f"\nAnalyzing {molecule_name}:")
    
    # Sort by number of flips for clearer output
    for data in sorted(combined_molecules[molecule_name], key=lambda x: x['n_flips']):
        n_flips = data['n_flips']
        df = data['df']
        
        # Filter out None values before getting unique values
        valid_weights = df['mol_weight'].dropna()
        valid_atoms = df['atoms'].dropna()
        
        unique_weights = valid_weights.unique()
        unique_atoms = valid_atoms.unique()
        
        print(f"\nFor {n_flips} flips:")
        print(f"Total molecules: {len(df)}")
        print(f"Valid molecules: {len(valid_weights)}")
        print(f"Unique molecular weights: {sorted(unique_weights)}")
        print(f"Unique atom counts: {sorted(unique_atoms)}")
        print(f"Number of unique weights: {len(unique_weights)}")
        print(f"Number of unique atom counts: {len(unique_atoms)}")


Analyzing parecoxib:

For 1 flips:
Total molecules: 2048
Valid molecules: 2048
Unique molecular weights: [370.09872805599997, 432.11437811999997, 451.120191772, 477.06644194800003, 522.161328312, 696.134855856]
Unique atom counts: [26, 31, 32, 38, 49]
Number of unique weights: 6
Number of unique atom counts: 5

For 2 flips:
Total molecules: 1024
Valid molecules: 1024
Unique molecular weights: [370.09872805599997, 432.11437811999997, 451.120191772, 477.06644194800003, 522.161328312, 696.134855856]
Unique atom counts: [26, 31, 32, 38, 49]
Number of unique weights: 6
Number of unique atom counts: 5

For 4 flips:
Total molecules: 512
Valid molecules: 512
Unique molecular weights: [370.09872805599997, 432.11437811999997, 451.120191772, 477.06644194800003, 505.097742076, 522.161328312, 527.1514919, 575.1514919, 696.134855856]
Unique atom counts: [26, 31, 32, 34, 38, 42, 49]
Number of unique weights: 9
Number of unique atom counts: 7

For 8 flips:
Total molecules: 256
Valid molecules: 255
Un

### Distribution of 1s and 0s when bits are flipped
---

In [123]:
def plot_bit_distributions(molecules_dict):
    """
    Creates two bar plots showing the distribution of 1s and 0s for each molecule and flip count.
    """
    # Collect data for plotting
    plot_data = {}
    flip_counts = set()
    
    # Data collection
    for molecule_name in molecules_dict:
        plot_data[molecule_name] = {'flips': [], 'ones_ratio': [], 'zeros_ratio': []}
        
        for data in sorted(molecules_dict[molecule_name], key=lambda x: x['n_flips']):
            n_flips = data['n_flips']
            df = data['df']
            
            total_bits = 0
            total_ones = 0
            
            for bits_str in df['FingerprintBitsFlipped']:
                try:
                    if isinstance(bits_str, str):
                        bits_str = bits_str.strip('[]')
                        bits = [int(x.strip()) for x in bits_str.split(',')]
                    else:
                        bits = bits_str
                    
                    total_bits += len(bits)
                    total_ones += sum(bits)
                except Exception:
                    continue
            
            if total_bits > 0:
                ones_ratio = (total_ones / total_bits) * 100
                zeros_ratio = ((total_bits - total_ones) / total_bits) * 100
                plot_data[molecule_name]['flips'].append(n_flips)
                plot_data[molecule_name]['ones_ratio'].append(ones_ratio)
                plot_data[molecule_name]['zeros_ratio'].append(zeros_ratio)
                flip_counts.add(n_flips)
    
    # Create two subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12))
    
    # Set up the bars
    bar_width = 0.15
    flip_counts = sorted(list(flip_counts))
    positions = np.arange(len(flip_counts))
    
    # Plot bars for each molecule in both subplots
    for i, (molecule, data) in enumerate(plot_data.items()):
        flip_to_pos = {flip: pos for pos, flip in enumerate(flip_counts)}
        pos = [flip_to_pos[flip] for flip in data['flips']]
        
        # Plot 1s distribution
        bars1 = ax1.bar([p + i * bar_width for p in pos], data['ones_ratio'], 
                       bar_width, label=molecule, alpha=0.8)
        
        # Add rounded percentage labels on top of bars for 1s
        for bar in bars1:
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(round(height))}%', ha='center', va='bottom')
        
        # Plot 0s distribution
        bars2 = ax2.bar([p + i * bar_width for p in pos], data['zeros_ratio'], 
                       bar_width, label=molecule, alpha=0.8)
        
        # Add rounded percentage labels on top of bars for 0s
        for bar in bars2:
            height = bar.get_height()
            ax2.text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(round(height))}%', ha='center', va='bottom')
    
    # Customize both plots
    for ax, title in zip([ax1, ax2], ['Distribution of 1s', 'Distribution of 0s']):
        ax.set_xlabel('Number of Flips')
        ax.set_ylabel('Percentage')
        ax.set_title(f'{title} Across Different Flip Counts')
        ax.set_xticks([p + bar_width * 2 for p in positions])
        ax.set_xticklabels(flip_counts)
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        ax.grid(True, linestyle='--', alpha=0.7)
    
    # Save the plot
    plt.tight_layout()
    plt.savefig(os.path.join(INPUT_DIR_1, 'bit_distributions.png'), 
                bbox_inches='tight', dpi=300)
    plt.close()

In [124]:
# Create the plots
plot_bit_distributions(combined_molecules)